In [153]:
import requests
import datetime as dt
import pandas as pd


def get_data(symbol, start_date=dt.date.today(), end_date=dt.date.today()):
    """
    """
    from io import StringIO
    symbol = symbol.upper()
    r = initial_request(symbol)
    (cookies, crumb) = get_cookies(r)
    data = final_request(symbol, start_date, end_date, cookies, crumb)
    return pd.read_csv(StringIO(data.text), index_col=0)


def initial_request(symbol):
    """
    """
    initial_url = "https://finance.yahoo.com/quote/" + symbol + "?p=" + symbol
    return requests.get(initial_url, verify=False)


def get_cookies(r):
    """
    """
    cookies = r.cookies
    crumbStore_begin = r.text.find("CrumbStore")
    crumbStore_end = r.text.find("}", crumbStore_begin) - 1
    crumbStore = r.text[crumbStore_begin:crumbStore_end]
    crumb_begin = crumbStore.rfind("\"") + 1
    crumb = crumbStore[crumb_begin:]
    return (cookies, crumb)
    

def date_utc(date):
    """
    """
    import calendar
    date = dt.datetime.combine(date, dt.datetime.min.time())
    return str(calendar.timegm(date.utctimetuple()))


def final_request(symbol, start_date, end_date, cookies, crumb):
    """
    """
    url = ("https://query1.finance.yahoo.com/v7/finance/download/"
           + symbol
           + "?period1="
           + date_utc(start_date)
           + "&period2="
           + date_utc(end_date)
           + "&interval=1d"
           + "&events=history"
           + "&crumb="
           + crumb)
    return requests.get(url, cookies=cookies, verify=False)

In [154]:
from pandas.tseries.offsets import *
get_data("GOOG", dt.date.today() - 7*BDay(), dt.date.today())

/Users/jmgate/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/jmgate/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-23,1159.849976,1171.626953,1158.750000,1169.969971,1169.969971,1333100
2018-01-24,1177.329956,1179.859985,1161.050049,1164.239990,1164.239990,1416600
2018-01-25,1172.530029,1175.939941,1162.760010,1170.369995,1170.369995,1480500
2018-01-26,1175.079956,1175.839966,1158.109985,1175.839966,1175.839966,2018800
2018-01-29,1176.479980,1186.890015,1171.979980,1175.579956,1175.579956,1378900
2018-01-30,1167.829956,1176.520020,1163.520020,1163.689941,1163.689941,1556300
2018-01-31,1170.569946,1173.000000,1159.130005,1169.939941,1169.939941,1512600
